In [1]:
import sys
from itertools import izip
from sklearn.cross_validation import KFold
import HTMLParser
import string

In [16]:
# Setup Train/Test Tweebo files to extract features

file_to_train = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/test1.txt"
file_to_train_tweebo = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/test2.txt"
result_file = "C:/Users/User/Documents/Cornell/Courses/NLP/HW4/result.txt"
kaggle = False
with open(file_to_train) as train:
    input_lines = train.readlines()
    labeled_tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                labeled_tweets.append(arr)
                arr = []
            else:
                if not kaggle:
                    (word, tag) = line.split()
                    arr.append((word, tag))
                else:
                    arr.append([line.strip()])
                
        except ValueError:
            continue

with open(file_to_train_tweebo) as train_tweebo:
    input_lines = train_tweebo.readlines()
    tweebo_tweets = []
    arr = []
    for line in input_lines:
        try:
            if line in ['\n', '\r\n']:
                tweebo_tweets.append(arr)
                arr = []
            else:
                (word_id, word, lemma, cpostag, postag, feats, head, dep_relation) = line.split()
                arr.append((word, word_id, postag, head, dep_relation));
        except ValueError:
            continue
            
features = []

for s1, s2 in zip(labeled_tweets, tweebo_tweets):
    tweebo_index = 0
    sentence = []
    head_diff = 0
    head_point = 0
    head_point_arr = []
    head_diff_arr = []
    raw_sentence = []
    for i in range(len(s1)):
        word1 = s1[i][0]
        escaped_word1 = HTMLParser.HTMLParser().unescape(s1[i][0]) 
        word2 = HTMLParser.HTMLParser().unescape(s2[tweebo_index][0]) 
        t_ind = tweebo_index
        raw_sentence.append(escaped_word1)
        if(word1 != word2 and escaped_word1 != word2):
            head_point = i+1
            head_point_arr.append(head_point)
            while True:
                if(word2 == word1 or word2 == escaped_word1):
                    break;
                else:
                    head_diff += 1
#                     if(int(s2[tweebo_index][3]) == -1 and int(s2[t_ind][3]) == -1):
#                         t_ind += 1
                    if((int(s2[tweebo_index][3]) != -1 and t_ind != tweebo_index)):
                        t_ind = tweebo_index
                    tweebo_index += 1
                    word2 += HTMLParser.HTMLParser().unescape(s2[tweebo_index][0])
            head_diff_arr.append(head_diff)
            
        if not kaggle:
            sentence.append([i+1, escaped_word1, s1[i][1], s2[t_ind][2], int(s2[t_ind][3]), s2[t_ind][4]])
        else:
            sentence.append([i+1, escaped_word1, s2[t_ind][2], int(s2[t_ind][3]), s2[t_ind][4]])
        tweebo_index += 1
    #head indexes reconciliation
    if len(head_diff_arr) > 0:
        for word in sentence:
            if not kaggle:
                old_head = word[4]
            else:
                old_head = word[3]
            if(old_head == -1 or old_head == 0):
                continue

            start = old_head - (len(s2) - len(s1)) - 2
            end = old_head + (len(s2) - len(s1))

            if(start < 0):
                start = 0
            if(end > len(sentence)):
                end = len(sentence)

            old_word = s2[old_head-1][0]
            next_word = ''
            if(old_head < len(s2)):
                next_word = s2[old_head][0]
            else:
                next_word = ''
            old_word_next = old_word + next_word
            
            if(old_head > 0):
                prev_word = s2[old_head-2][0]
            else:
                prev_word = ''
                
            prev_old_word = prev_word + old_word
            
            new_word = ''

            for i in xrange(start, end):
                if(HTMLParser.HTMLParser().unescape(old_word) == sentence[i][1] \
                   or HTMLParser.HTMLParser().unescape(old_word_next) in sentence[i][1]\
                  or HTMLParser.HTMLParser().unescape(prev_old_word) in sentence[i][1]):
                    new_word = sentence[i]

            if not kaggle:
                word[4] = new_word[0]
            else:
                word[3] = new_word[0]
    features.append(sentence)
    
open(result_file, 'w').close()
with open (result_file, 'w') as f: 
    for tweet in features:
        for word in tweet:
            f.write(' '.join(str(x) for x in word).strip())
            f.write("\n")
        f.write("\n")